## Save activation values

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('donkeycar')
sys.path.append('src')


In [2]:
from tensorflow import keras
import multiprocessing
import numpy as np
from losses import get_pipe, get_dataset_from_pipe
from donkeycar.utils import get_model_by_type
from dataloader import load_records
import donkeycar as dk
import matplotlib.pyplot as plt


________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.5 ...



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [3]:
cfg = dk.load_config(config_path='../mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

cfg.SEQUENCE_LENGTH = 3
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
cfg.ROI_CROP_KEEP_SIZE = False
N_FOLDS = None

model_type = '3d-modified-steering'

loading config file: ../mycar/config.py
loading personal config over-rides from myconfig.py


In [4]:

DATA_PATH = '../data/'

fast_data_path1 = 'faster-autodrive-18_20'
fast_data_path2 = 'fastest-autodrive-18_40'

fast_speed_data1 = load_records(fast_data_path1, cfg, DATA_PATH, TUBRECORD_SIZE)

fast_speed_data2 = load_records(fast_data_path2, cfg, DATA_PATH, TUBRECORD_SIZE)

fast_speed_data = fast_speed_data1

fast_speed_data.extend(fast_speed_data2)
len(fast_speed_data)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['../data/faster-autodrive-18_20']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/ood/../data/faster-autodrive-18_20/catalog_14.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['../data/fastest-autodrive-18_40']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/ood/../data/fastest-autodrive-18_40/catalog_10.catalog


20771

In [5]:
slow_data_path = 'daytime/daytime-manual-slower-2'
slow_speed_data = load_records(slow_data_path, cfg, DATA_PATH, TUBRECORD_SIZE)
len(slow_speed_data)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['../data/daytime/daytime-manual-slower-2']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/ood/../data/daytime/daytime-manual-slower-2/catalog_21.catalog


20642

In [6]:
fast_model_path = '../models/daytime/3d-steering-faster-daytime-full-data-multiply.h5'
fast_model = get_model_by_type(model_type, cfg)
fast_model.load(fast_model_path)
fast_model.interpreter.model.summary()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
2022-07-11 00:16:02.105835: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model ../models/daytime/3d-steering-faster-daytime-full-data-multiply.h5
INFO:donkeycar.parts.interpreter:Loading model ../models/daytime/3d-steering-faster-daytime-full-data-multiply.h5


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_in (InputLayer)         [(None, 3, 60, 160, 3)]   0         
                                                                 
 conv3d (Conv3D)             (None, 1, 20, 53, 16)     1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 1, 10, 26, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 1, 10, 26, 32)     4640      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 5, 13, 32)     0         
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 1, 5, 13, 32)      9248  

In [7]:
list(map(lambda x: x.name, fast_model.interpreter.model.layers))

['img_in',
 'conv3d',
 'max_pooling3d',
 'conv3d_1',
 'max_pooling3d_1',
 'conv3d_2',
 'flatten',
 'dense',
 'batch_normalization',
 'activation',
 'dropout',
 'dense_1',
 'batch_normalization_1',
 'activation_1',
 'dropout_1',
 'outputs']

In [8]:
fast_model_intermediate_layer = keras.Model(fast_model.interpreter.model.input,
                                            fast_model.interpreter.model.get_layer('dense_1').output)
pipe = get_pipe(fast_model, cfg, fast_speed_data)
data = get_dataset_from_pipe(pipe)
fast_model_fast_ai_data = fast_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

 12/163 [=>............................] - ETA: 4:01

KeyboardInterrupt: 

In [ ]:
fast_model_intermediate_layer_after_bn = keras.Model(fast_model.interpreter.model.input,
                                            fast_model.interpreter.model.get_layer('batch_normalization_1').output)
pipe = get_pipe(fast_model, cfg, fast_speed_data)
data = get_dataset_from_pipe(pipe)
fast_model_fast_ai_data_after_bn = fast_model_intermediate_layer_after_bn.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
pipe = get_pipe(fast_model, cfg, slow_speed_data)
data = get_dataset_from_pipe(pipe)
fast_model_slow_manual_data = fast_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
pipe = get_pipe(fast_model, cfg, slow_speed_data)
data = get_dataset_from_pipe(pipe)
fast_model_slow_manual_data_after_bn = fast_model_intermediate_layer_after_bn.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
slow_model_path = '../models/daytime/3d-steering-slower-daytime-full-data-multiply.h5'
slow_model = get_model_by_type(model_type, cfg)
slow_model.load(slow_model_path)
slow_model.interpreter.model.summary()

In [ ]:
slow_model_intermediate_layer = keras.Model(slow_model.interpreter.model.input,
                                            slow_model.interpreter.model.get_layer('dense_1').output)

In [ ]:
slow_model_intermediate_layer_after_bn = keras.Model(slow_model.interpreter.model.input,
                                            slow_model.interpreter.model.get_layer('batch_normalization_1').output)


In [ ]:
pipe = get_pipe(slow_model, cfg, fast_speed_data)
data = get_dataset_from_pipe(pipe)
slow_model_fast_ai_data = slow_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
pipe = get_pipe(slow_model, cfg, fast_speed_data)
data = get_dataset_from_pipe(pipe)
slow_model_fast_ai_data_after_bn = slow_model_intermediate_layer_after_bn.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
pipe = get_pipe(slow_model, cfg, slow_speed_data)
data = get_dataset_from_pipe(pipe)
slow_model_slow_manual_data = slow_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
pipe = get_pipe(slow_model, cfg, slow_speed_data)
data = get_dataset_from_pipe(pipe)
slow_model_slow_manual_data_after_bn = slow_model_intermediate_layer_after_bn.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
np.savetxt("fast_model_fast_ai_data.csv.gz", np.float16(fast_model_fast_ai_data))
np.savetxt("fast_model_slow_manual_data.csv.gz", np.float16(fast_model_slow_manual_data))
np.savetxt("slow_model_fast_ai_data.csv.gz", np.float16(slow_model_fast_ai_data))
np.savetxt("slow_model_slow_manual_data.csv.gz", np.float16(slow_model_slow_manual_data))

# Analyze activation values

In [ ]:
fast_model_fast_ai_data = np.loadtxt("fast_model_fast_ai_data.csv.gz")
fast_model_slow_manual_data = np.loadtxt("fast_model_slow_manual_data.csv.gz")
slow_model_fast_ai_data = np.loadtxt("slow_model_fast_ai_data.csv.gz")
slow_model_slow_manual_data = np.loadtxt("slow_model_slow_manual_data.csv.gz")

### Sanity check dimensions

In [ ]:
[d.shape for d in (fast_model_fast_ai_data, fast_model_slow_manual_data, slow_model_fast_ai_data, slow_model_slow_manual_data)]

In [ ]:
[d.shape for d in (fast_model_fast_ai_data_after_bn, fast_model_slow_manual_data_after_bn, slow_model_fast_ai_data_after_bn, slow_model_slow_manual_data_after_bn)]

In [ ]:
fast_model_fast_ai_data_neuron_means = np.mean(np.abs(fast_model_fast_ai_data), axis=0)
fast_model_slow_manual_data_neuron_means = np.mean(np.abs(fast_model_slow_manual_data), axis=0)
slow_model_fast_ai_data_neuron_means = np.mean(np.abs(slow_model_fast_ai_data), axis=0)
slow_model_slow_manual_data_neuron_means = np.mean(np.abs(slow_model_slow_manual_data), axis=0)

fast_model_fast_ai_data_neuron_std = np.std(fast_model_fast_ai_data, axis=0)
fast_model_slow_manual_data_neuron_std = np.std(fast_model_slow_manual_data, axis=0)
slow_model_fast_ai_data_neuron_std = np.std(slow_model_fast_ai_data, axis=0)
slow_model_slow_manual_data_neuron_std = np.std(slow_model_slow_manual_data, axis=0)

fast_model_fast_ai_data_neuron_means.mean(), fast_model_slow_manual_data_neuron_means.mean(), slow_model_fast_ai_data_neuron_means.mean(), slow_model_slow_manual_data_neuron_means.mean()

In [ ]:
fast_model_fast_ai_data_after_bn_neuron_means = np.mean(np.abs(fast_model_fast_ai_data_after_bn), axis=0)
fast_model_slow_manual_data_after_bn_neuron_means = np.mean(np.abs(fast_model_slow_manual_data_after_bn), axis=0)
slow_model_fast_ai_data_after_bn_neuron_means = np.mean(np.abs(slow_model_fast_ai_data_after_bn), axis=0)
slow_model_slow_manual_data_after_bn_neuron_means = np.mean(np.abs(slow_model_slow_manual_data_after_bn), axis=0)

fast_model_fast_ai_data_after_bn_neuron_std = np.std(fast_model_fast_ai_data_after_bn, axis=0)
fast_model_slow_manual_data_after_bn_neuron_std = np.std(fast_model_slow_manual_data_after_bn, axis=0)
slow_model_fast_ai_data_after_bn_neuron_std = np.std(slow_model_fast_ai_data_after_bn, axis=0)
slow_model_slow_manual_data_after_bn_neuron_std = np.std(slow_model_slow_manual_data_after_bn, axis=0)

fast_model_fast_ai_data_after_bn_neuron_means.mean(), fast_model_slow_manual_data_after_bn_neuron_means.mean(), slow_model_fast_ai_data_after_bn_neuron_means.mean(), slow_model_slow_manual_data_after_bn_neuron_means.mean()

In [ ]:
fast_model_fast_ai_data_neuron_std.mean(), fast_model_slow_manual_data_neuron_std.mean(), slow_model_fast_ai_data_neuron_std.mean(), slow_model_slow_manual_data_neuron_std.mean()

In [ ]:
fast_model_fast_ai_data_after_bn_neuron_std.mean(), fast_model_slow_manual_data_after_bn_neuron_std.mean(), slow_model_fast_ai_data_after_bn_neuron_std.mean(), slow_model_slow_manual_data_after_bn_neuron_std.mean()


In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='all', sharey='all', figsize=(20, 15))
ax1.plot(fast_model_fast_ai_data_neuron_means)
ax1.set_title('Fast model Fast AI data')
ax2.plot(fast_model_slow_manual_data_neuron_means)
ax2.set_title('Fast model Slow manual data')
ax3.plot(slow_model_fast_ai_data_neuron_means)
ax3.set_title('Slow model Fast AI data')
ax4.plot(slow_model_slow_manual_data_neuron_means)
ax4.set_title('Slow model Slow manual data')

for ax in fig.get_axes():
    ax.set(xlabel='Neuron indices', ylabel='Absolute mean of neuron activations')
    ax.label_outer()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='all', sharey='all', figsize=(20, 15))
ax1.plot(fast_model_fast_ai_data_neuron_std)
ax1.set_title('Fast model Fast AI data')
ax2.plot(fast_model_slow_manual_data_neuron_std)
ax2.set_title('Fast model Slow manual data')
ax3.plot(slow_model_fast_ai_data_neuron_std)
ax3.set_title('Slow model Fast AI data')
ax4.plot(slow_model_slow_manual_data_neuron_std)
ax4.set_title('Slow model Slow manual data')

for ax in fig.get_axes():
    ax.set(xlabel='Neuron indices', ylabel='Standard deviation of neuron activations')
    ax.label_outer()


## Use Manual fast data instead AI-collected fast data

In [ ]:
fast_data_path = 'daytime/daytime-manual-faster'
fast_speed_data = load_records(fast_data_path, cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_speed_data)

In [ ]:
pipe = get_pipe(fast_model, cfg, fast_speed_data)
data = get_dataset_from_pipe(pipe)
fast_model_fast_manual_data = fast_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
pipe = get_pipe(slow_model, cfg, fast_speed_data)
data = get_dataset_from_pipe(pipe)
slow_model_fast_manual_data = slow_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
np.savetxt("fast_model_fast_manual_data.csv.gz", np.float16(fast_model_fast_manual_data))
np.savetxt("slow_model_fast_manual_data.csv.gz", np.float16(slow_model_fast_manual_data))


In [ ]:
fast_model_fast_manual_data = np.loadtxt("fast_model_fast_manual_data.csv.gz")
slow_model_fast_manual_data = np.loadtxt("slow_model_fast_manual_data.csv.gz")

### Sanity check dimensions

In [ ]:
[d.shape for d in (fast_model_fast_manual_data, fast_model_slow_manual_data, slow_model_fast_manual_data, slow_model_slow_manual_data)]


In [ ]:
fast_model_fast_manual_data_neuron_means = np.mean(np.abs(fast_model_fast_manual_data), axis=0)
fast_model_slow_manual_data_neuron_means = np.mean(np.abs(fast_model_slow_manual_data), axis=0)
slow_model_fast_manual_data_neuron_means = np.mean(np.abs(slow_model_fast_manual_data), axis=0)
slow_model_slow_manual_data_neuron_means = np.mean(np.abs(slow_model_slow_manual_data), axis=0)

fast_model_fast_manual_data_neuron_std = np.std(fast_model_fast_manual_data, axis=0)
fast_model_slow_manual_data_neuron_std = np.std(fast_model_slow_manual_data, axis=0)
slow_model_fast_manual_data_neuron_std = np.std(slow_model_fast_manual_data, axis=0)
slow_model_slow_manual_data_neuron_std = np.std(slow_model_slow_manual_data, axis=0)

fast_model_fast_manual_data_neuron_means.mean(), fast_model_slow_manual_data_neuron_means.mean(), slow_model_fast_manual_data_neuron_means.mean(), slow_model_slow_manual_data_neuron_means.mean()

In [ ]:
fast_model_fast_manual_data_neuron_std.mean(), fast_model_slow_manual_data_neuron_std.mean(), slow_model_fast_manual_data_neuron_std.mean(), slow_model_slow_manual_data_neuron_std.mean()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='all', sharey='all', figsize=(20, 15))
ax1.plot(fast_model_fast_manual_data_neuron_means)
ax1.set_title('Fast model Fast manual data')
ax2.plot(fast_model_slow_manual_data_neuron_means)
ax2.set_title('Fast model Slow manual data')
ax3.plot(slow_model_fast_manual_data_neuron_means)
ax3.set_title('Slow model Fast manual data')
ax4.plot(slow_model_slow_manual_data_neuron_means)
ax4.set_title('Slow model Slow manual data')

for ax in fig.get_axes():
    ax.set(xlabel='Neuron indices', ylabel='Absolute mean of neuron activations')
    ax.label_outer()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='all', sharey='all', figsize=(20, 15))
ax1.plot(fast_model_fast_manual_data_neuron_std)
ax1.set_title('Fast model Fast manual data')
ax2.plot(fast_model_slow_manual_data_neuron_std)
ax2.set_title('Fast model Slow manual data')
ax3.plot(slow_model_fast_manual_data_neuron_std)
ax3.set_title('Slow model Fast manual data')
ax4.plot(slow_model_slow_manual_data_neuron_std)
ax4.set_title('Slow model Slow manual data')

for ax in fig.get_axes():
    ax.set(xlabel='Neuron indices', ylabel='Standard deviation of neuron activations')
    ax.label_outer()


## Use fast and slow AI data, both on night-time

In [ ]:
fast_data_path = 'faster-autodrive'
fast_speed_data = load_records(fast_data_path, cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_speed_data)

In [ ]:
pipe = get_pipe(fast_model, cfg, fast_speed_data)
data = get_dataset_from_pipe(pipe)
fast_model_fast_ai_nighttime_data = fast_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
pipe = get_pipe(fast_model, cfg, fast_speed_data)
data = get_dataset_from_pipe(pipe)
slow_model_fast_ai_nighttime_data = slow_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
slow_data_path = 'slower-autodrive'
slow_speed_data = load_records(slow_data_path, cfg, DATA_PATH, TUBRECORD_SIZE)
len(slow_speed_data)

In [ ]:
pipe = get_pipe(fast_model, cfg, slow_speed_data)
data = get_dataset_from_pipe(pipe)
fast_model_slow_ai_nighttime_data = fast_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
pipe = get_pipe(fast_model, cfg, slow_speed_data)
data = get_dataset_from_pipe(pipe)
slow_model_slow_ai_nighttime_data = slow_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [ ]:
np.savetxt("fast_model_fast_ai_nighttime_data.csv.gz", np.float16(fast_model_fast_ai_nighttime_data))
np.savetxt("fast_model_slow_ai_nighttime_data.csv.gz", np.float16(fast_model_slow_ai_nighttime_data))
np.savetxt("slow_model_fast_ai_nighttime_data.csv.gz", np.float16(slow_model_fast_ai_nighttime_data))
np.savetxt("slow_model_slow_ai_nighttime_data.csv.gz", np.float16(slow_model_slow_ai_nighttime_data))


In [ ]:
fast_model_fast_ai_nighttime_data = np.loadtxt("fast_model_fast_ai_nighttime_data.csv.gz")
fast_model_slow_ai_nighttime_data = np.loadtxt("fast_model_slow_ai_nighttime_data.csv.gz")
slow_model_fast_ai_nighttime_data = np.loadtxt("slow_model_fast_ai_nighttime_data.csv.gz")
slow_model_slow_ai_nighttime_data = np.loadtxt("slow_model_slow_ai_nighttime_data.csv.gz")


### Sanity check dimensions

In [ ]:
[d.shape for d in (fast_model_fast_ai_nighttime_data, fast_model_slow_ai_nighttime_data, slow_model_fast_ai_nighttime_data, slow_model_slow_ai_nighttime_data)]


In [ ]:
fast_model_fast_ai_nighttime_data_neuron_means = np.mean(np.abs(fast_model_fast_ai_nighttime_data), axis=0)
fast_model_slow_ai_nighttime_data_neuron_means = np.mean(np.abs(fast_model_slow_ai_nighttime_data), axis=0)
slow_model_fast_ai_nighttime_data_neuron_means = np.mean(np.abs(slow_model_fast_ai_nighttime_data), axis=0)
slow_model_slow_ai_nighttime_data_neuron_means = np.mean(np.abs(slow_model_slow_ai_nighttime_data), axis=0)
fast_model_fast_ai_nighttime_data_neuron_std = np.std(fast_model_fast_ai_nighttime_data, axis=0)
fast_model_slow_ai_nighttime_data_neuron_std = np.std(fast_model_slow_ai_nighttime_data, axis=0)
slow_model_fast_ai_nighttime_data_neuron_std = np.std(slow_model_fast_ai_nighttime_data, axis=0)
slow_model_slow_ai_nighttime_data_neuron_std = np.std(slow_model_slow_ai_nighttime_data, axis=0)

fast_model_fast_ai_nighttime_data_neuron_means.mean(), fast_model_slow_ai_nighttime_data_neuron_means.mean(), slow_model_fast_ai_nighttime_data_neuron_means.mean(), slow_model_slow_ai_nighttime_data_neuron_means.mean()

In [ ]:
fast_model_fast_ai_nighttime_data_neuron_std.mean(), fast_model_slow_ai_nighttime_data_neuron_std.mean(), slow_model_fast_ai_nighttime_data_neuron_std.mean(), slow_model_slow_ai_nighttime_data_neuron_std.mean()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='all', sharey='all', figsize=(20, 15))
ax1.plot(fast_model_fast_ai_nighttime_data_neuron_means)
ax1.set_title('Fast model Fast AI nighttime data')
ax2.plot(fast_model_slow_ai_nighttime_data_neuron_means)
ax2.set_title('Fast model Slow AI nighttime data')
ax3.plot(slow_model_fast_ai_nighttime_data_neuron_means)
ax3.set_title('Slow model Fast AI nighttime data')
ax4.plot(slow_model_slow_ai_nighttime_data_neuron_means)
ax4.set_title('Slow model Slow AI nighttime data')

for ax in fig.get_axes():
    ax.set(xlabel='Neuron indices', ylabel='Absolute mean of neuron activations')
    ax.label_outer()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='all', sharey='all', figsize=(20, 15))
ax1.plot(fast_model_fast_ai_nighttime_data_neuron_std)
ax1.set_title('Fast model Fast AI nighttime data')
ax2.plot(fast_model_slow_ai_nighttime_data_neuron_std)
ax2.set_title('Fast model Slow AI nighttime data')
ax3.plot(slow_model_fast_ai_nighttime_data_neuron_std)
ax3.set_title('Slow model Fast AI nighttime data')
ax4.plot(slow_model_slow_ai_nighttime_data_neuron_std)
ax4.set_title('Slow model Slow AI nighttime data')

for ax in fig.get_axes():
    ax.set(xlabel='Neuron indices', ylabel='Standard deviation of neuron activations')
    ax.label_outer()

## Compare daytime vs nighttime activations

### Daytime

In [ ]:
fast_model_fast_ai_data_neuron_means.mean(), slow_model_fast_ai_data_neuron_means.mean()


In [ ]:
fast_model_fast_ai_data_neuron_std.mean(), slow_model_fast_ai_data_neuron_std.mean()


### Nighttime

In [ ]:
fast_model_fast_ai_nighttime_data_neuron_means.mean(), slow_model_fast_ai_nighttime_data_neuron_means.mean()


In [ ]:
fast_model_fast_ai_nighttime_data_neuron_std.mean(), slow_model_fast_ai_nighttime_data_neuron_std.mean()


In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='all', sharey='all', figsize=(20, 15))
ax1.plot(fast_model_fast_ai_data_neuron_means)
ax1.set_title('Fast model Fast AI daytime')
ax2.plot(fast_model_fast_ai_nighttime_data_neuron_means)
ax2.set_title('Fast model Slow AI nighttime')
ax3.plot(slow_model_fast_ai_data_neuron_means)
ax3.set_title('Slow model Fast AI daytime')
ax4.plot(slow_model_slow_ai_nighttime_data_neuron_means)
ax4.set_title('Slow model Slow AI nighttime')

for ax in fig.get_axes():
    ax.set(xlabel='Neuron indices', ylabel='Mean of neuron activations')
    ax.label_outer()

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='all', sharey='all', figsize=(20, 15))
ax1.plot(fast_model_fast_ai_data_neuron_std)
ax1.set_title('Fast model Fast AI daytime')
ax2.plot(fast_model_fast_ai_nighttime_data_neuron_std)
ax2.set_title('Fast model Slow AI nighttime')
ax3.plot(slow_model_fast_ai_data_neuron_std)
ax3.set_title('Slow model Fast AI daytime')
ax4.plot(slow_model_slow_ai_nighttime_data_neuron_std)
ax4.set_title('Slow model Slow AI nighttime')

for ax in fig.get_axes():
    ax.set(xlabel='Neuron indices', ylabel='Standard deviation of neuron activations')
    ax.label_outer()

# Retrained models to be sure on what data they were trained on:

Fast AI Daytime model from data in the "faster-autodrive-18_20" and "fastest-autodrive-18_40" folders
Fast AI Nighttime model from data in the "faster-autodrive" folders.

### Daytime data

In [24]:
DATA_PATH = '../data/'

fast_speed_data1 = load_records('faster-autodrive-18_20', cfg, DATA_PATH, TUBRECORD_SIZE)
fast_speed_data2 = load_records('fastest-autodrive-18_40', cfg, DATA_PATH, TUBRECORD_SIZE)

fast_daytime_ai_traindata = fast_speed_data1
fast_daytime_ai_traindata.extend(fast_speed_data2)
len(fast_daytime_ai_traindata)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['../data/faster-autodrive-18_20']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/ood/../data/faster-autodrive-18_20/catalog_14.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['../data/fastest-autodrive-18_40']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/ood/../data/fastest-autodrive-18_40/catalog_10.catalog


20771

In [20]:
fast_nighttime_ai_traindata = load_records('faster-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_nighttime_ai_traindata)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['../data/faster-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/ood/../data/faster-autodrive/catalog_20.catalog


19250

### Retrained daytime model

In [11]:
fast_ai_daytime_model_path = 'retrain/full-faster-daytime-autodrive-3dcnn.h5'
fast_ai_daytime_model = get_model_by_type(model_type, cfg)
fast_ai_daytime_model.load(fast_ai_daytime_model_path)
fast_ai_daytime_model.interpreter.model.summary()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model retrain/full-faster-daytime-autodrive-3dcnn.h5
INFO:donkeycar.parts.interpreter:Loading model retrain/full-faster-daytime-autodrive-3dcnn.h5


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_in (InputLayer)         [(None, 3, 60, 160, 3)]   0         
                                                                 
 conv3d (Conv3D)             (None, 1, 20, 53, 16)     1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 1, 10, 26, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 1, 10, 26, 32)     4640      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 5, 13, 32)     0         
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 1, 5, 13, 32)      9248  

### Daytime model daytime data

In [ ]:
fast_ai_daytime_model_intermediate_layer = keras.Model(fast_ai_daytime_model.interpreter.model.input,
                                                       fast_ai_daytime_model.interpreter.model.get_layer('dense_1').output)
pipe = get_pipe(fast_ai_daytime_model, cfg, fast_daytime_ai_traindata)
data = get_dataset_from_pipe(pipe)
fast_ai_daytime_model_daytime_data = fast_ai_daytime_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

In [16]:
fast_ai_daytime_model_daytime_data.shape

(20864, 256)

### Daytime model nighttime data

In [33]:
pipe = get_pipe(fast_ai_daytime_model, cfg, fast_nighttime_ai_traindata)
data = get_dataset_from_pipe(pipe)
fast_ai_daytime_model_nighttime_data = fast_ai_daytime_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

151/151 [==============================] - 35s 231ms/step


In [34]:
fast_ai_daytime_model_nighttime_data.shape

(19328, 256)

### Retrained nighttime model

In [19]:
fast_ai_nighttime_model_path = 'retrain/full-faster-nighttime-autodrive-3dcnn.h5'
fast_ai_nighttime_model = get_model_by_type(model_type, cfg)
fast_ai_nighttime_model.load(fast_ai_nighttime_model_path)
fast_ai_nighttime_model.interpreter.model.summary()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model retrain/full-faster-nighttime-autodrive-3dcnn.h5
INFO:donkeycar.parts.interpreter:Loading model retrain/full-faster-nighttime-autodrive-3dcnn.h5


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_in (InputLayer)         [(None, 3, 60, 160, 3)]   0         
                                                                 
 conv3d (Conv3D)             (None, 1, 20, 53, 16)     1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 1, 10, 26, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 1, 10, 26, 32)     4640      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 5, 13, 32)     0         
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 1, 5, 13, 32)      9248  

In [26]:
fast_ai_nighttime_model_intermediate_layer = keras.Model(fast_ai_nighttime_model.interpreter.model.input,
                                                         fast_ai_nighttime_model.interpreter.model.get_layer('dense_1').output)

### Nighttime model daytime data

In [27]:

pipe = get_pipe(fast_ai_nighttime_model, cfg, fast_daytime_ai_traindata)
data = get_dataset_from_pipe(pipe)
fast_ai_nighttime_model_daytime_data = fast_ai_nighttime_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

163/163 [==============================] - 197s 1s/step


In [32]:
fast_ai_nighttime_model_daytime_data.shape

(20864, 256)

### Nighttime model nighttime data

In [28]:

pipe = get_pipe(fast_ai_nighttime_model, cfg, fast_nighttime_ai_traindata)
data = get_dataset_from_pipe(pipe)
fast_ai_nighttime_model_nighttime_data = fast_ai_nighttime_model_intermediate_layer.predict(
    data,
    workers=multiprocessing.cpu_count(),
    use_multiprocessing=True,
    steps=len(pipe),
    verbose=1)

151/151 [==============================] - 165s 1s/step


In [31]:
fast_ai_nighttime_model_nighttime_data.shape

(19328, 256)

## Save/load

In [35]:
np.savetxt("retrain/fast_ai_daytime_model_daytime_data.csv.gz", np.float16(fast_ai_daytime_model_daytime_data))
np.savetxt("retrain/fast_ai_daytime_model_nighttime_data.csv.gz", np.float16(fast_ai_daytime_model_nighttime_data))
np.savetxt("retrain/fast_ai_nighttime_model_daytime_data.csv.gz", np.float16(fast_ai_nighttime_model_daytime_data))
np.savetxt("retrain/fast_ai_nighttime_model_nighttime_data.csv.gz", np.float16(fast_ai_nighttime_model_nighttime_data))


## Analyze

In [36]:
fast_ai_daytime_model_daytime_data_neuron_means = np.mean(np.abs(fast_ai_daytime_model_daytime_data), axis=0)
fast_ai_daytime_model_nighttime_data_neuron_means = np.mean(np.abs(fast_ai_daytime_model_nighttime_data), axis=0)
fast_ai_nighttime_model_daytime_data_neuron_means = np.mean(np.abs(fast_ai_nighttime_model_daytime_data), axis=0)
fast_ai_nighttime_model_nighttime_data_neuron_means = np.mean(np.abs(fast_ai_nighttime_model_nighttime_data), axis=0)

fast_ai_daytime_model_daytime_data_neuron_std = np.std(fast_ai_daytime_model_daytime_data, axis=0)
fast_ai_daytime_model_nighttime_data_neuron_std = np.std(fast_ai_daytime_model_nighttime_data, axis=0)
fast_ai_nighttime_model_daytime_data_neuron_std = np.std(fast_ai_nighttime_model_daytime_data, axis=0)
fast_ai_nighttime_model_nighttime_data_neuron_std = np.std(fast_ai_nighttime_model_nighttime_data, axis=0)

fast_ai_daytime_model_daytime_data_neuron_means.mean(), fast_ai_daytime_model_nighttime_data_neuron_means.mean(), fast_ai_nighttime_model_daytime_data_neuron_means.mean(), fast_ai_nighttime_model_nighttime_data_neuron_means.mean()

(0.6727674, 0.5097982, 0.48071343, 0.3933084)

In [37]:
fast_ai_daytime_model_daytime_data_neuron_std.mean(), fast_ai_daytime_model_nighttime_data_neuron_std.mean(), fast_ai_nighttime_model_daytime_data_neuron_std.mean(), fast_ai_nighttime_model_nighttime_data_neuron_std.mean()


(0.66178465, 0.36294407, 0.4789427, 0.39016736)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='all', sharey='all', figsize=(20, 15))
ax1.plot(fast_model_fast_ai_data_neuron_std)
ax1.set_title('Fast model Fast AI daytime')
ax2.plot(fast_model_fast_ai_nighttime_data_neuron_std)
ax2.set_title('Fast model Slow AI nighttime')
ax3.plot(slow_model_fast_ai_data_neuron_std)
ax3.set_title('Slow model Fast AI daytime')
ax4.plot(slow_model_slow_ai_nighttime_data_neuron_std)
ax4.set_title('Slow model Slow AI nighttime')

for ax in fig.get_axes():
    ax.set(xlabel='Neuron indices', ylabel='Standard deviation of neuron activations')
    ax.label_outer()

